# Perform SQL query in natural Language

In this notebook, we provide an example of using SQL agent to perform sql query in natural lanugage.

In [ ]:
# First init the agent config.
import agentscope
from agentscope.agents.user_agent import UserAgent
from conversation_sql import SQLAgent

agentscope.init(
    model_configs="./configs/model_configs.json",
)

In [ ]:
# Then provide the database in sql_lite format
from sql_utils import create_sqlite_db_from_schema


example_db_id = "concert_singer"
db_schema_path = "./database/concert_singer/schema.sql"
db_sqlite_path = "./database/concert_singer/concert_singer.sqlite"
create_sqlite_db_from_schema(db_schema_path, db_sqlite_path)


In [ ]:
# We init the SQL agent and user agent
sql_agent = SQLAgent(
    name="SQL Agent",
    db_id=example_db_id,
    db_path=db_sqlite_path,
    model_config_name="gpt-4",
)
user_agent = UserAgent()

# Now we can interact with SQL agent
mss = None
while True:
    mss = sql_agent(mss)
    mss = user_agent(mss)

    if mss.content == "exit":
        print("Exiting the conversation.")
        break

We show the following dialog as an example.

SQL Agent: Hi, I am an agent able to preform SQL querys base on natual language instructions. 

Below is a description of the database concert_singer provided.

The database consists of four tables:

1. Stadium: This table holds records related to stadiums. Each record in this table represents a particular stadium, identified by a unique Stadium_ID. Other details about a stadium, such as its Location, Name, Capacity, Highest, Lowest, and Average, are also stored in this table.

2. Singer: This provides information about different singers. Each row in this table corresponds to one singer and is uniquely identified by Singer_ID. The table also stores other information about a singer like Name, Country, Song_Name, Song_release_year, Age, and a Boolean Is_male field indicating if the singer is male.

3. Concert: This table stores data about various concerts. Each concert is identified by a unique concert_ID. The table also has concert_Name, Theme, Stadium_ID where the concert occurred, and Year fields. The Stadium_ID field here is a foreign key that references the Stadium_ID from the Stadium table, indicating which stadium the concert took place in.

4. Singer_in_Concert: This table is built to establish a relationship between the Singer and Concert tables. It has a composite primary key consisting of concert_ID and Singer_ID. These are foreign keys that link to respective entries in the Concert and Singer tables. It signifies which singer performed in which concert.

Is there any you want to ask about the database concert_singer?

User: Show all countries and the number of singers in each country.

SQL Agent: Generated SQL query is: SELECT Country , COUNT(*) FROM singer GROUP BY Country

 The execution result is: [('France', 4), ('Netherlands', 1), ('United States', 1)]

The number of singers in each country is as follows:
- France: 4 singers
- Netherlands: 1 singer
- United States: 1 singer

User: How many singers have held concert in multiple stadiums?Give me the name of these singers.

SQL Agent: Generated SQL query is: SELECT T1.Name FROM singer AS T1 JOIN singer_in_concert AS T2 ON T1.Singer_ID = T2.Singer_ID JOIN concert AS T3 ON T2.concert_ID = T3.concert_ID GROUP BY T1.Name HAVING COUNT(DISTINCT T3.Stadium_ID) > 1 

 The execution result is: [('John Nizinik',), ('Justin Brown',), ('Timbaland',), ('Tribal King',)]
 
There are 4 singers who have held concerts in multiple stadiums: John Nizinik, Justin Brown, Timbaland, and Tribal King.

User: Thank you.

SQL Agent: Your message doesn't contain a question. Could you please provide more details or ask a question?

User: How many football players do we have?

SQL Agent: No, this question is not appropriate for the given SQL schema. The existing schema does not contain any table or field related to football players. It deals with concert, singer and stadium information only.

User: eixt

SQL Agent: I'm sorry for the confusion, but "eixt" does not seem to be a question or a query related to the given SQL schema. It appears to be a typo. If you have a specific question about the SQL schema or need help with a query, feel free to ask!

User: exit

Exiting the conversation.